In [29]:
from utils import load_dataset_to_numpy

In [30]:
trian_dataset = r'datasets\cat_dog_dataset_train.csv'
test_dataset = r"datasets\cat_dog_dataset_test.csv"

X_train, y_train = load_dataset_to_numpy(trian_dataset)
X_test, y_test = load_dataset_to_numpy(trian_dataset)

In [31]:
print ("Number of training examples: " + str(X_train.shape[0]))
print ("Number of testing examples: " + str(X_test.shape[0]))
print ("Height/Width of each image: " + str(X_train[1].shape[0]))
print ("Each image is of size: (" + str(X_train[1].shape[0]) + ", " + str(X_train[1].shape[0]) + ", 3)")
print ("train_set_x shape: " + str(X_train.shape))
print ("train_set_y shape: " + str(y_train.shape))
print ("test_set_x shape: " + str(X_test.shape))
print ("test_set_y shape: " + str(y_test.shape))

Number of training examples: 557
Number of testing examples: 557
Height/Width of each image: 64
Each image is of size: (64, 64, 3)
train_set_x shape: (557, 64, 64, 3)
train_set_y shape: (557,)
test_set_x shape: (557, 64, 64, 3)
test_set_y shape: (557,)


#### Flatten the arrays into a single vector of shape (height * width * 3, 1)

In [40]:
train_set_x_flatten = X_train.reshape(X_train.shape[0], -1).T
test_set_x_flatten = X_test.reshape(X_test.shape[0], -1).T
print("Train and test shape after flattening:")
print(f"train: {train_set_x_flatten.shape} \ntest: {test_set_x_flatten.shape}")

Train and test shape after flattening:
train: (12288, 557) 
test: (12288, 557)
